In [2]:
import argparse
import tqdm
import time

import torch
import torch.nn.functional as F

import Qtransformer

import ROOT
from root_numpy import root2array, rec2array
import logging
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


Welcome to JupyROOT 6.22/06


In [3]:
def readConfig(configFile="config_qqyy.json"):
    with open("config/" + configFile, "r") as f_obj:
        config = json.load(f_obj)
        logging.info("Signal sample(s) %s", config["signal_file_list"])
        logging.info("Background sample(s) %s", config["bkg_file_list"])
        logging.info("Signal(s) tree name %s", config["signal_tree_name"])
        logging.info("Background(s) tree name %s", config["bkg_tree_name"])
        logging.info("Signal(s) weight variable %s", config["signal_weight_name"])
        logging.info("Background(s) weight variable %s", config["bkg_weight_name"])
        logging.info("Signal(s) variables %s", config["var_signal"])
        logging.info("Background(s) variables %s", config["var_bkg"])

    return config


In [4]:
def Trandform(sigArray, bkgArray, rangConst="_0_1"):
    sigConstrain = []
    bkgConstrain = []

    mini1 = np.min(sigArray, axis=0)
    mini2 = np.min(bkgArray, axis=0)
    maxi1 = np.max(sigArray, axis=0)
    maxi2 = np.max(bkgArray, axis=0)
    mini = np.minimum(mini1, mini2)
    maxi = np.maximum(maxi1, maxi2)

    if rangConst == "_0_1":
        sigConstrain = (sigArray - np.min(sigArray, axis=0)) / np.ptp(sigArray, axis=0)
    elif rangConst == "_n1_1":
        sigConstrain = (2.*(sigArray - mini)/(maxi-mini))-1
        bkgConstrain = (2.*(bkgArray - mini)/(maxi-mini))-1
    elif rangConst == "_0_2pi":
        sigConstrain = (2 * np.pi * (sigArray - np.min(sigArray, axis=0))) / np.ptp(sigArray, axis=0)
    elif rangConst == "_npi_pi":
        sigConstrain = -np.pi + np.pi * (sigArray - np.min(sigArray, axis=0)) / np.ptp(sigArray, axis=0)

    return sigConstrain, bkgConstrain


In [5]:
def preparingData(confiFile="config_qqyy_8.json", prossEvent=100, fraction=0.5, seed=None, dataType="Classical"):
    config = readConfig(confiFile)

    signal_dataset = root2array(
        filenames=config["signal_file_list"],
        treename=config["signal_tree_name"],
        branches=config["var_signal"],
        selection=config["signal_selection"],
        include_weight=False,
        weight_name="weight",
        stop=prossEvent,
    )

    bkg_dataset = root2array(
        filenames=config["bkg_file_list"],
        treename=config["bkg_tree_name"],
        branches=config["var_bkg"],
        selection=config["bkg_selection"],
        include_weight=False,
        weight_name="weight",
        stop=prossEvent,
    )

    signal_dataset = rec2array(signal_dataset)
    bkg_dataset = rec2array(bkg_dataset)

    signal_dataset, bkg_dataset = Trandform(signal_dataset, bkg_dataset, "_n1_1")

    train_size = int(len(signal_dataset) * fraction)
    test_size = int(len(signal_dataset) * fraction)

    X_signal = signal_dataset
    X_background = bkg_dataset

    y_signal = np.ones(X_signal.shape[0])
    y_background = np.ones(X_background.shape[0])
    y_background = -1 * y_background

    X = np.concatenate([X_signal, X_background], axis=0)
    y = np.concatenate([y_signal, y_background])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, train_size=train_size, test_size=test_size, random_state=seed
    )

    if dataType=="Classical":
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        X = scaler.fit_transform(X)
    else:
        print("The data will be prepared for the quantum case so no transformation is needed.")

    labels = {1: "S", -1: "B"}

    train_dataset = {
        labels[1]: X_train[y_train == 1],
        labels[-1]: X_train[y_train == -1],
    }
    test_dataset = {labels[1]: X_test[y_test == 1], labels[-1]: X_test[y_test == -1]}

    return (
        X_train,
        X_test,
        y_train,
        y_test,
        train_dataset,
        test_dataset,
        X,
        y,
    )


In [6]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc


def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    for batch in iterator:
        optimizer.zero_grad()

        inputs = torch.LongTensor(batch.text[0])
        if inputs.size(1) > MAX_SEQ_LEN:
            inputs = inputs[:, :MAX_SEQ_LEN]
        predictions = model(inputs).squeeze(1)
        
        label = batch.label - 1
        loss = criterion(predictions, label)
        acc = binary_accuracy(predictions, label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [7]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    with torch.no_grad():
        for batch in iterator:
            inputs = torch.LongTensor(batch.text[0])
            if inputs.size(1) > MAX_SEQ_LEN:
                inputs = inputs[:, :MAX_SEQ_LEN]
            predictions = model(inputs).squeeze(1)
            
            label = batch.label - 1
            loss = criterion(predictions, label)
            acc = binary_accuracy(predictions, label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)


In [8]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


In [12]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-D', '--q_device', default='local', type=str)
    #parser.add_argument('-B', '--batch_size', default=32, type=int)
    #parser.add_argument('-E', '--n_epochs', default=5, type=int)
    #parser.add_argument('-C', '--n_classes', default=2, type=int)
    #parser.add_argument('-l', '--lr', default=0.001, type=float)
    #parser.add_argument('-v', '--vocab_size', default=20000, type=int)
    #parser.add_argument('-e', '--embed_dim', default=8, type=int)
    #parser.add_argument('-s', '--max_seq_len', default=64, type=int)
    #parser.add_argument('-f', '--ffn_dim', default=8, type=int)
    #parser.add_argument('-t', '--n_transformer_blocks', default=1, type=int)
    #parser.add_argument('-H', '--n_heads', default=2, type=int)
    #parser.add_argument('-q', '--n_qubits_transformer', default=0, type=int)
    #parser.add_argument('-Q', '--n_qubits_ffn', default=0, type=int)
    #parser.add_argument('-L', '--n_qlayers', default=1, type=int)
    #parser.add_argument('-d', '--dropout_rate', default=0.1, type=float)
    args = parser.parse_args()

   # MAX_SEQ_LEN = args.max_seq_len

   # train_data, test_data, y_train, y_test, train_dataset, test_dataset, X, y = preparingData()
   # TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
   # LABEL = data.LabelField(dtype=torch.float)
   # fields = [('text', TEXT), ('label', LABEL)]
   # train_examples = []
   # test_examples = []

IndentationError: unexpected indent (<ipython-input-12-f52976424435>, line 2)

In [9]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('-D', '--q_device', default='local', type=str)
    parser.add_argument('-B', '--batch_size', default=32, type=int)
    parser.add_argument('-E', '--n_epochs', default=5, type=int)
    parser.add_argument('-C', '--n_classes', default=2, type=int)
    parser.add_argument('-l', '--lr', default=0.001, type=float)
    parser.add_argument('-v', '--vocab_size', default=20000, type=int)
    parser.add_argument('-e', '--embed_dim', default=8, type=int)
    parser.add_argument('-s', '--max_seq_len', default=64, type=int)
    parser.add_argument('-f', '--ffn_dim', default=8, type=int)
    parser.add_argument('-t', '--n_transformer_blocks', default=1, type=int)
    parser.add_argument('-H', '--n_heads', default=2, type=int)
    parser.add_argument('-q', '--n_qubits_transformer', default=0, type=int)
    parser.add_argument('-Q', '--n_qubits_ffn', default=0, type=int)
    parser.add_argument('-L', '--n_qlayers', default=1, type=int)
    parser.add_argument('-d', '--dropout_rate', default=0.1, type=float)
    args = parser.parse_args()

    MAX_SEQ_LEN = args.max_seq_len

    train_data, test_data, y_train, y_test, train_dataset, test_dataset, X, y = preparingData()
    TEXT = data.Field(lower=True, include_lengths=True, batch_first=True)
    LABEL = data.LabelField(dtype=torch.float)    
    fields = [('text', TEXT), ('label', LABEL)]
    train_examples = []
    test_examples = []
    
    for label, data in train_dataset.items():
        examples = [data[i] for i in range(len(data))]
        train_examples += [(example, label) for example in examples]
    
    for label, data in test_dataset.items():
        examples = [data[i] for i in range(len(data))]
        test_examples += [(example, label) for example in examples]
    
    train_data = data.Dataset(train_examples, fields)
    test_data = data.Dataset(test_examples, fields)
    
    TEXT.build_vocab(train_data, max_size=args.vocab_size - 2)
    LABEL.build_vocab(train_data)

    train_iter, test_iter = data.BucketIterator.splits((train_data, test_data), batch_size=args.batch_size)
    
    model = TextClassifier(embed_dim=args.embed_dim,
                           num_heads=args.n_heads,
                           num_blocks=args.n_transformer_blocks,
                           num_classes=args.n_classes,
                           vocab_size=args.vocab_size,
                           ffn_dim=args.ffn_dim,
                           n_qubits_transformer=args.n_qubits_transformer,
                           n_qubits_ffn=args.n_qubits_ffn,
                           n_qlayers=args.n_qlayers,
                           dropout=args.dropout_rate,
                           q_device=args.q_device)
    print(f'The model has {count_parameters(model):,} trainable parameters')

    optimizer = torch.optim.Adam(lr=args.lr, params=model.parameters())
    if args.n_classes < 3:
        criterion = torch.nn.BCEWithLogitsLoss()
    else:
        criterion = torch.nn.CrossEntropyLoss()

    best_valid_loss = float('inf')
    for iepoch in range(args.n_epochs):
        start_time = time.time()

        print(f"Epoch {iepoch+1}/{args.n_epochs}")

        train_loss, train_acc = train(model, train_iter, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, test_iter, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'model.pt')
        
        print(f'Epoch: {iepoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')


usage: ipykernel_launcher.py [-h] [-D Q_DEVICE] [-B BATCH_SIZE] [-E N_EPOCHS]
                             [-C N_CLASSES] [-l LR] [-v VOCAB_SIZE]
                             [-e EMBED_DIM] [-s MAX_SEQ_LEN] [-f FFN_DIM]
                             [-t N_TRANSFORMER_BLOCKS] [-H N_HEADS]
                             [-q N_QUBITS_TRANSFORMER] [-Q N_QUBITS_FFN]
                             [-L N_QLAYERS] [-d DROPOUT_RATE]
ipykernel_launcher.py: error: argument -f/--ffn_dim: invalid int value: '/home/sqy/.local/share/jupyter/runtime/kernel-v2-119651NN13sEmQpA2.json'
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/sqy/anaconda3/envs/Qistik/lib/python3.8/argparse.py", line 2422, in _get_value
    result = type_func(arg_string)
ValueError: invalid literal for int() with base 10: '/home/sqy/.local/share/jupyter/runtime/kernel-v2-119651NN13sEmQpA2.json'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/sqy/anaconda3/envs/Qistik/lib/python3.8/argparse.py", line 1800, in parse_known_args
    namespace, args = self._parse_known_args(args, namespace)
  File "/home/sqy/anaconda3/envs/Qistik/lib/python3.8/argparse.py", line 2006, in _parse_known_args
    start_index = consume_optional(start_index)
  File "/home/sqy/anaconda3/envs/Qistik/lib/python3.8/argparse.py", line 1946, in consume_optional
    take_action(action, args, option_string)
  File "/home/sqy/anaconda3/envs/Qistik/lib/python3.8/argparse.py", line 1858, in take_action
    argument_values = self._get_values(action, argument_str

TypeError: object of type 'NoneType' has no len()